# Distinguishing muffins and Chihuahuas with NNs

The aim of this project is to train a neural network for the binary classification of muffins and Chihuahuas based on the images contained in the relative [Kaggle dataset](https://www.kaggle.com/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification).

We import the Yelp dataset from Kaggle, using a token.

In [40]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

os.environ['KAGGLE_USERNAME'] = "luciaannamellini"
os.environ['KAGGLE_KEY'] = "c209fcf223ecdd6be8fd373196354f4b"

api = KaggleApi()
api.authenticate()
api.dataset_download_files("samuelcortinhas/muffin-vs-chihuahua-image-classification")

KeyboardInterrupt: 

In [38]:
FOLDERNAME = "muffin-vs-chihuahua-image-classification"
PATH = "./"

In [39]:
import zipfile

with zipfile.ZipFile(PATH + FOLDERNAME + ".zip", 'r') as zip_ref:
    zip_ref.extractall(PATH)
os.remove(PATH + FOLDERNAME +".zip")

BadZipFile: Bad magic number for file header

If already present we import, or otherwise we install, the `tensorflow`package. Above all functionalities we will be using the `keras` module to construct the desired neural network for this project.

In [ ]:
import importlib
import pip
def import_or_install(package_name, alias=None):
    try:
        imported_package = importlib.import_module(package_name)
        if alias:
            globals()[alias] = imported_package
        print(f"{package_name} is already installed.")
    except ImportError:
        print(f"{package_name} is not installed. Installing...")
        pip.install(package_name)
        if alias:
            globals()[alias] = importlib.import_module(package_name)
        print(f"{package_name} has been installed.")


import_or_install("tensorflow","tf")

tensorflow is already installed.


In [ ]:
from PIL import Image
def remove_badly_encoded_images(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.JPG'):
            try:
                img = Image.open(directory+filename)  # open the image file
                img.verify()  # verify that it is, in fact an image
            except (IOError, SyntaxError):
                print(f"Removing file {filename}")
                os.remove(directory+filename)

deleted = remove_badly_encoded_images(PATH + FOLDERNAME)
print("Deleted images:",deleted)

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: './muffin-vs-chihuahua-image-classification/'

In [ ]:
print("Training set:")
train_ds = tf.keras.utils.image_dataset_from_directory(foldername+"/train", label_mode = "categorical", batch_size = None)
print("Test set:")
test_ds = tf.keras.utils.image_dataset_from_directory(foldername+"/test", label_mode = "categorical", batch_size = None)

Training set:


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 36: invalid continuation byte

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

NameError: name 'train_ds' is not defined

<Figure size 720x720 with 0 Axes>